In [2]:
import pandas as pd
pd.options.display.max_columns = 999
import pgeocode as geo
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [3]:
sc= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='1997')
sc.head(1)

,fips,areaname,bowl97,civic97,fitnes97,golf97,relig97,sport97,memspt97,pol97,prof97,bus97,labor97,memnec97,respn00,pvote96,pop97,nccs97,assn97,sk97
0,1001,"Autauga, AL",0,4,0,0,35,1,1,0,0,4,5,0,0.63,0.559864,41238,94.0,1.212474,-0.684649


In [4]:
cols_rename= {
            'areaname':'area',
            'County_Name':'area',
            'fitnes97': 'fit97'
            }

In [5]:
sc.describe()

,fips,bowl97,civic97,fitnes97,golf97,relig97,sport97,memspt97,pol97,prof97,bus97,labor97,memnec97,respn00,pvote96,pop97,nccs97,assn97,sk97
count,3108.000000,3108.00000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3.108000e+03,3100.000000,3108.000000,3.108000e+03
mean,30685.814672,1.76834,11.563707,3.576898,1.953668,50.919562,0.631274,4.800515,0.581403,2.517053,4.426319,6.034106,2.376448,0.626091,0.549201,8.713573e+04,353.943226,1.422432,2.579463e-07
std,14982.940191,3.95514,30.815900,12.015822,3.858427,106.488553,2.910832,11.801004,2.654460,10.505383,14.084029,18.112391,5.470392,0.088242,0.097136,2.838173e+05,1058.445841,0.664457,1.438265e+00
min,1001.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.102008,7.800000e+01,1.000000,0.000000,-4.310725e+00
25%,19046.500000,0.00000,1.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.570000,0.479125,1.121750e+04,50.000000,1.000843,-9.961086e-01
50%,29214.000000,1.00000,3.000000,1.000000,1.000000,22.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.630000,0.547212,2.433400e+04,108.000000,1.329816,-2.336947e-01
75%,46009.500000,2.00000,10.000000,2.000000,2.000000,48.000000,0.000000,4.000000,0.000000,1.000000,3.000000,4.000000,2.000000,0.690000,0.612464,6.093275e+04,252.000000,1.711658,7.578042e-01
max,56045.000000,91.00000,698.000000,315.000000,70.000000,2669.000000,81.000000,199.000000,53.000000,255.000000,310.000000,353.000000,131.000000,0.840000,1.020520,9.206538e+06,28823.000000,7.299270,8.240644e+00


In [6]:
sc.rename(columns=cols_rename, inplace=True)
sc.columns= sc.columns.str.strip().str.lower()
sc.columns=sc.columns.str.replace('\d+','',regex=True).str.strip()
sc['nccs']= sc['nccs'].replace(np.nan, 0) # This is one of the key things that make results: Before this fix: corr= 0.54. After this fix corr= 0.99

sc['nccs']= sc['nccs']/(sc['pop']/1000)
sc['assn']= (sc['relig'] + sc['civic'] + sc['bus'] + sc['pol'] + sc['prof'] + sc['labor'] + sc['bowl'] + sc['fit'] + sc['golf'] + sc['sport']) / (sc['pop']/1000)

sc.drop(columns=['bowl', 'civic', 'fit', 'golf', 'relig', 'sport', 'pol', 'prof', 'bus', 'labor'], inplace=True)
sc.drop(columns=['memspt','memnec'], inplace=True)
sc.drop(columns=['pop'],inplace=True)
sc.head(3)

,fips,area,respn,pvote,nccs,assn,sk
0,1001,"Autauga, AL",0.63,0.559864,2.279451,1.188224,-0.684649
1,1003,"Baldwin, AL",0.54,0.528056,3.103777,1.375188,-0.739966
2,1005,"Barbour, AL",0.54,0.482278,3.589653,0.809432,-1.388539


In [7]:
x = sc.loc[:, ['respn','pvote','assn','nccs']].values
# Separating out the target
y = sc.loc[:,['fips']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1'])
pca_df.head(2)

,pc1
0,-0.607770
1,-0.708665


In [8]:
sc['pc1']=pca_df[['pc1']]

In [9]:
sc.head()

,fips,area,respn,pvote,nccs,assn,sk,pc1
0,1001,"Autauga, AL",0.63,0.559864,2.279451,1.188224,-0.684649,-0.607770
1,1003,"Baldwin, AL",0.54,0.528056,3.103777,1.375188,-0.739966,-0.708665
2,1005,"Barbour, AL",0.54,0.482278,3.589653,0.809432,-1.388539,-1.358385
3,1007,"Bibb, AL",0.51,0.501432,2.269445,1.495771,-1.058240,-1.002602
4,1009,"Blount, AL",0.61,0.485385,1.615131,0.871321,-1.541663,-1.483992


In [10]:
sc['sk'].corr(sc['pc1'])

0.9969667909381396

In [11]:
sc.shape

(3108, 8)

In [12]:
sc['year']=1997
sc.to_csv("output\\sc97.csv")